# GROMACS molecular dynamics demonstration

Molecular dynamics (MD) is a versatile method for simulating the time-dependent behavior of molecular systems. This notebook demonstrates how MD simulations can be run using the GROMACS software to study the self-assembly of lipids. To accelerate the simulations, a coarse-grained (CG) model using the Martini force field is employed. In addition, parallel processing with multiple CPU cores is leveraged to demonstrate the principles of high-performance computing (HPC).

In [140]:
!export OMP_NUM_THREADS=1

Let's try inserting 50 dipalmitoylphosphatidylcholine (DPPC) lipids into a cubic simulation box with a side length of 6 Å. The resulting system will be stored in a file named `lipids.gro`.

In [141]:
!gmx_mpi insert-molecules -o lipids -ci ff22/DPPC.gro -nmol 50 -try 200 -box 6.0

                 :-) GROMACS - gmx insert-molecules, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

Next, we need to fill the `lipids.gro` system with water (solvate). The resulting system will be stored in the file `solv.gro`.

In [142]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/W400.gro -o solv -scale 2.0

                     :-) GROMACS - gmx solvate, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Check the previous outputs for the number of lipids and water molecules that were *de facto* added. We need to modify a template topology file with these amounts, respectively.

In [143]:
!sed "s|FIXME|$PWD|g" template/template.top > topol.top
!echo DPPC 50 >> topol.top
!echo W 1140 >> topol.top

Now, let's copy a template recipe for performing *energy minimization*

In [144]:
!cp template/em.mdp .
!gmx_mpi grompp -f em -c solv -p topol -o em -maxwarn 10

                      :-) GROMACS - gmx grompp, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

In [145]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -v -s em

                      :-) GROMACS - gmx mdrun, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Step=  102, Dmax= 5.9e-02 nm, Epot= -3.68549e+04 Fmax= 5.51836e+02, atom= 1699
Step=  103, Dmax= 7.1e-02 nm, Epot= -3.68925e+04 Fmax= 8.69357e+02, atom= 1699
Step=  104, Dmax= 8.5e-02 nm, Epot= -3.69313e+04 Fmax= 5.77809e+02, atom= 1581
Step=  106, Dmax= 5.1e-02 nm, Epot= -3.69821e+04 Fmax= 1.63446e+02, atom= 1718
Step=  107, Dmax= 6.2e-02 nm, Epot= -3.70211e+04 Fmax= 1.15456e+03, atom= 1699
Step=  108, Dmax= 7.4e-02 nm, Epot= -3.70964e+04 Fmax= 8.44004e+02, atom= 1699
Step=  109, Dmax= 8.9e-02 nm, Epot= -3.71257e+04 Fmax= 5.27207e+02, atom= 1581
Step=  111, Dmax= 5.3e-02 nm, Epot= -3.71739e+04 Fmax= 1.81131e+02, atom= 1718
Step=  113, Dmax= 3.2e-02 nm, Epot= -3.72134e+04 Fmax= 5.52789e+02, atom= 797
Step=  114, Dmax= 3.8e-02 nm, Epot= -3.72440e+04 Fmax= 2.84979e+02, atom= 823
Step=  115, Dmax= 4.6e-02 nm, Epot= -3.72654e+04 Fmax= 1.00587e+03, atom= 797
Step=  116, Dmax= 5.5e-02 nm, Epot= -3.72997e+04 Fmax= 3.02436e+02, atom= 797
Step=  118, Dmax= 3.3e-02 nm, Epot= -3.73325e+04 Fmax= 1

Step=  300, Dmax= 3.0e-02 nm, Epot= -3.93065e+04 Fmax= 3.08627e+02, atom= 1533
Step=  301, Dmax= 3.6e-02 nm, Epot= -3.93089e+04 Fmax= 3.16478e+02, atom= 1533
Step=  303, Dmax= 2.2e-02 nm, Epot= -3.93205e+04 Fmax= 4.92323e+01, atom= 1738
Step=  305, Dmax= 1.3e-02 nm, Epot= -3.93275e+04 Fmax= 1.51124e+02, atom= 1534
Step=  306, Dmax= 1.6e-02 nm, Epot= -3.93324e+04 Fmax= 1.48236e+02, atom= 1533
Step=  307, Dmax= 1.9e-02 nm, Epot= -3.93360e+04 Fmax= 1.72745e+02, atom= 1534
Step=  308, Dmax= 2.3e-02 nm, Epot= -3.93390e+04 Fmax= 2.34113e+02, atom= 1533
Step=  309, Dmax= 2.7e-02 nm, Epot= -3.93425e+04 Fmax= 2.34381e+02, atom= 1534
Step=  310, Dmax= 3.3e-02 nm, Epot= -3.93426e+04 Fmax= 3.67614e+02, atom= 1534
Step=  311, Dmax= 3.9e-02 nm, Epot= -3.93472e+04 Fmax= 3.07583e+02, atom= 1534
Step=  313, Dmax= 2.3e-02 nm, Epot= -3.93580e+04 Fmax= 6.16663e+01, atom= 1738
Step=  315, Dmax= 1.4e-02 nm, Epot= -3.93623e+04 Fmax= 1.89569e+02, atom= 1534
Step=  316, Dmax= 1.7e-02 nm, Epot= -3.93686e+04 Fma

Step=  435, Dmax= 1.4e-02 nm, Epot= -3.97336e+04 Fmax= 1.31511e+02, atom= 1534
Step=  436, Dmax= 1.7e-02 nm, Epot= -3.97356e+04 Fmax= 1.66764e+02, atom= 1534
Step=  437, Dmax= 2.0e-02 nm, Epot= -3.97377e+04 Fmax= 2.02263e+02, atom= 1534
Step=  438, Dmax= 2.4e-02 nm, Epot= -3.97392e+04 Fmax= 2.25799e+02, atom= 1534
Step=  439, Dmax= 2.9e-02 nm, Epot= -3.97394e+04 Fmax= 3.20203e+02, atom= 1534
Step=  440, Dmax= 3.5e-02 nm, Epot= -3.97413e+04 Fmax= 2.93978e+02, atom= 1533
Step=  442, Dmax= 2.1e-02 nm, Epot= -3.97496e+04 Fmax= 5.39745e+01, atom= 1534
Step=  444, Dmax= 1.3e-02 nm, Epot= -3.97522e+04 Fmax= 1.94666e+02, atom= 1534
Step=  445, Dmax= 1.5e-02 nm, Epot= -3.97568e+04 Fmax= 8.39192e+01, atom= 1534
Step=  446, Dmax= 1.8e-02 nm, Epot= -3.97578e+04 Fmax= 2.39759e+02, atom= 1534
Step=  447, Dmax= 2.2e-02 nm, Epot= -3.97625e+04 Fmax= 1.48813e+02, atom= 1534
Step=  449, Dmax= 1.3e-02 nm, Epot= -3.97657e+04 Fmax= 8.10118e+01, atom= 1533
Step=  450, Dmax= 1.6e-02 nm, Epot= -3.97677e+04 Fma

In [146]:
%%bash
cat > ndx_inp <<EOF
0 & !2
name 4 SOLVENTS
q
EOF

In [147]:
!gmx_mpi make_ndx -f confout.gro < ndx_inp

                     :-) GROMACS - gmx make_ndx, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu M

In [148]:
!cp template/md.mdp .
!sed -i "s/ADDINDEXGROUPS/DPPC SOLVENTS/" md.mdp

In [149]:
!gmx_mpi grompp -f md.mdp -c confout -p topol.top -maxwarn 10 -n index -o md

                      :-) GROMACS - gmx grompp, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

In [150]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

                      :-) GROMACS - gmx mdrun, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

imb F  1% step 8600, remaining wall clock time:    22 s          
imb F  3% step 8700, remaining wall clock time:    22 s          
imb F  3% step 8800, remaining wall clock time:    22 s          
imb F  1% step 8900, remaining wall clock time:    22 s          
imb F  3% step 9000, remaining wall clock time:    22 s          
imb F  3% step 9100, remaining wall clock time:    22 s          
imb F  4% step 9200, remaining wall clock time:    22 s          
imb F  3% step 9300, remaining wall clock time:    22 s          
imb F  1% step 9400, remaining wall clock time:    21 s          
imb F  3% step 9500, remaining wall clock time:    21 s          
imb F  9% step 9600, remaining wall clock time:    21 s          
imb F  1% step 9700, remaining wall clock time:    21 s          
imb F  2% step 9800, remaining wall clock time:    21 s          
imb F  3% step 9900, remaining wall clock time:    21 s          
imb F  4% step 10000, remaining wall clock time:    21 s          
imb F  3%

imb F  2% step 21100, remaining wall clock time:    19 s          
imb F  4% step 21200, remaining wall clock time:    19 s          
imb F  2% step 21300, remaining wall clock time:    19 s          
imb F  1% step 21400, remaining wall clock time:    19 s          
imb F  3% step 21500, remaining wall clock time:    19 s          
imb F  2% step 21600, remaining wall clock time:    19 s          
imb F  2% step 21700, remaining wall clock time:    19 s          
imb F  1% step 21800, remaining wall clock time:    19 s          
imb F  1% step 21900, remaining wall clock time:    19 s          
imb F  3% step 22000, remaining wall clock time:    19 s          
imb F  2% step 22100, remaining wall clock time:    19 s          
imb F  2% step 22200, remaining wall clock time:    19 s          
imb F  2% step 22300, remaining wall clock time:    19 s          
imb F  2% step 22400, remaining wall clock time:    19 s          
imb F  1% step 22500, remaining wall clock time:    19 s      

imb F  2% step 33500, remaining wall clock time:    17 s          
imb F  3% step 33600, remaining wall clock time:    17 s          
imb F  3% step 33700, remaining wall clock time:    17 s          
imb F  3% step 33800, remaining wall clock time:    17 s          
imb F  2% step 33900, remaining wall clock time:    17 s          
imb F  3% step 34000, remaining wall clock time:    17 s          
imb F  1% step 34100, remaining wall clock time:    17 s          
imb F 22% step 34200, remaining wall clock time:    17 s          
imb F  1% step 34300, remaining wall clock time:    17 s          
imb F  3% step 34400, remaining wall clock time:    17 s          
imb F  2% step 34500, remaining wall clock time:    17 s          
imb F  1% step 34600, remaining wall clock time:    17 s          
imb F  2% step 34700, remaining wall clock time:    17 s          
imb F  2% step 34800, remaining wall clock time:    17 s          
imb F  2% step 34900, remaining wall clock time:    17 s      

imb F  6% step 45900, remaining wall clock time:    15 s          
imb F  2% step 46000, remaining wall clock time:    15 s          
imb F  1% step 46100, remaining wall clock time:    15 s          
imb F  2% step 46200, remaining wall clock time:    15 s          
imb F  1% step 46300, remaining wall clock time:    15 s          
imb F  1% step 46400, remaining wall clock time:    15 s          
imb F  0% step 46500, remaining wall clock time:    15 s          
imb F  2% step 46600, remaining wall clock time:    15 s          
imb F  2% step 46700, remaining wall clock time:    15 s          
imb F  1% step 46800, remaining wall clock time:    15 s          
imb F  4% step 46900, remaining wall clock time:    15 s          
imb F  2% step 47000, remaining wall clock time:    15 s          
imb F  1% step 47100, remaining wall clock time:    15 s          
imb F  2% step 47200, remaining wall clock time:    15 s          
imb F  1% step 47300, remaining wall clock time:    15 s      

...

imb F  1% step 95400, remaining wall clock time:     8 s          
imb F  2% step 95500, remaining wall clock time:     8 s          
imb F  0% step 95600, remaining wall clock time:     7 s          
imb F  5% step 95700, remaining wall clock time:     7 s          
imb F  3% step 95800, remaining wall clock time:     7 s          
imb F  1% step 95900, remaining wall clock time:     7 s          
imb F  0% step 96000, remaining wall clock time:     7 s          
imb F  2% step 96100, remaining wall clock time:     7 s          
imb F  1% step 96200, remaining wall clock time:     7 s          
imb F  0% step 96300, remaining wall clock time:     7 s          
imb F  3% step 96400, remaining wall clock time:     7 s          
imb F  1% step 96500, remaining wall clock time:     7 s          
imb F  2% step 96600, remaining wall clock time:     7 s          
imb F  2% step 96700, remaining wall clock time:     7 s          
imb F  3% step 96800, remaining wall clock time:     7 s      

imb F  1% step 107800, remaining wall clock time:     6 s          
imb F  2% step 107900, remaining wall clock time:     6 s          
imb F  1% step 108000, remaining wall clock time:     6 s          
imb F  4% step 108100, remaining wall clock time:     6 s          
imb F  1% step 108200, remaining wall clock time:     6 s          
imb F  1% step 108300, remaining wall clock time:     6 s          
imb F  1% step 108400, remaining wall clock time:     6 s          
imb F  1% step 108500, remaining wall clock time:     6 s          
imb F  1% step 108600, remaining wall clock time:     6 s          
imb F  3% step 108700, remaining wall clock time:     6 s          
imb F  2% step 108800, remaining wall clock time:     6 s          
imb F  2% step 108900, remaining wall clock time:     5 s          
imb F  1% step 109000, remaining wall clock time:     5 s          
imb F  2% step 109100, remaining wall clock time:     5 s          
imb F  1% step 109200, remaining wall clock time

imb F  2% step 119900, remaining wall clock time:     4 s          
imb F  1% step 120000, remaining wall clock time:     4 s          
imb F  2% step 120100, remaining wall clock time:     4 s          
imb F  1% step 120200, remaining wall clock time:     4 s          
imb F  2% step 120300, remaining wall clock time:     4 s          
imb F  2% step 120400, remaining wall clock time:     4 s          
imb F  1% step 120500, remaining wall clock time:     4 s          
imb F  1% step 120600, remaining wall clock time:     4 s          
imb F  2% step 120700, remaining wall clock time:     4 s          
imb F  1% step 120800, remaining wall clock time:     4 s          
imb F  1% step 120900, remaining wall clock time:     4 s          
imb F  1% step 121000, remaining wall clock time:     4 s          
imb F  2% step 121100, remaining wall clock time:     4 s          
imb F  1% step 121200, remaining wall clock time:     4 s          
imb F  1% step 121300, remaining wall clock time

imb F  1% step 132100, remaining wall clock time:     2 s          
imb F  1% step 132200, remaining wall clock time:     2 s          
imb F  1% step 132300, remaining wall clock time:     2 s          
imb F  1% step 132400, remaining wall clock time:     2 s          
imb F  2% step 132500, remaining wall clock time:     2 s          
imb F  1% step 132600, remaining wall clock time:     2 s          
imb F  1% step 132700, remaining wall clock time:     2 s          
imb F  2% step 132800, remaining wall clock time:     2 s          
imb F  2% step 132900, remaining wall clock time:     2 s          
imb F  1% step 133000, remaining wall clock time:     2 s          
imb F  1% step 133100, remaining wall clock time:     2 s          
imb F  2% step 133200, remaining wall clock time:     2 s          
imb F  1% step 133300, remaining wall clock time:     2 s          
imb F  2% step 133400, remaining wall clock time:     2 s          
imb F  2% step 133500, remaining wall clock time

imb F  3% step 144400, remaining wall clock time:     0 s          
imb F  3% step 144500, remaining wall clock time:     0 s          
imb F  2% step 144600, remaining wall clock time:     0 s          
imb F 22% step 144700, remaining wall clock time:     0 s          
imb F  1% step 144800, remaining wall clock time:     0 s          
imb F  0% step 144900, remaining wall clock time:     0 s          
imb F  0% step 145000, remaining wall clock time:     0 s          
imb F  9% step 145100, remaining wall clock time:     0 s          
imb F  1% step 145200, remaining wall clock time:     0 s          
imb F  2% step 145300, remaining wall clock time:     0 s          
imb F  2% step 145400, remaining wall clock time:     0 s          
imb F  2% step 145500, remaining wall clock time:     0 s          
imb F  2% step 145600, remaining wall clock time:     0 s          
imb F  2% step 145700, remaining wall clock time:     0 s          
imb F  1% step 145800, remaining wall clock time

In [151]:
!echo 0 > trj_inp
!gmx_mpi trjconv -s md -f md.xtc -o md_whole.xtc -pbc mol < trj_inp

                     :-) GROMACS - gmx trjconv, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Cleanup

In [153]:
!rm -f *gro* *trr *xtc *inp *top *tpr *mdp *edr *cpt *log* *ndx